In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyTreeInspector
using Gurobi
import StochasticOptimization
using Plots; gr()
using JLD2
using ProgressMeter

In [ ]:
import LCPSim
import LearningMPC
import Hoppers
import Nets

In [ ]:
robot = Hoppers.Hopper()
mechanism = robot.mechanism
xstar = Hoppers.nominal_state(robot)

basevis = Visualizer()[:hopper]
setgeometry!(basevis, robot)
settransform!(basevis[:robot], xstar)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=20,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, TimeLimit=120, MIPGap=1e-1, FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Q, R = Hoppers.default_costs(robot)
foot = findbody(mechanism, "foot")
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt, [Point3D(default_frame(foot), 0., 0., 0.)])
lqrsol.S .= 1 ./ mpc_params.Δt .* Q

hidden_widths = [16, 16, 16]
activation = Nets.elu
net = LearningMPC.control_net(mechanism, hidden_widths, activation)

net_controller = x -> Nets.predict(net, state_vector(x))

mpc_controller = LearningMPC.MPCController(mechanism, 
    robot.environment, mpc_params, lqrsol, 
    [net_controller, lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}()
playback_sink = LearningMPC.PlaybackSink(basevis[:robot], mpc_params.Δt)
live_viewer = LearningMPC.live_viewer(mechanism, basevis[:robot])

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
    playback_sink,
#     (args...) -> println("tick")
#     (x, results) -> live_viewer(x)
)


dagger_controller = LearningMPC.call_each(
    LearningMPC.dagger_controller(
        mpc_controller,
        net_controller,
        0.2),
    live_viewer
    )

termination = x -> false

dataset = LearningMPC.Dataset(lqrsol)

updater = Nets.adam_updater(net)

gradient_sensitivity = 0.2
loss = LearningMPC.sensitive_loss(net, gradient_sensitivity)
adam_opts = Nets.AdamOpts(learning_rate=1e-3, batch_size=1)


x0 = MechanismState{Float64}(mechanism)

x_init = MechanismState{Float64}(mechanism)
set_configuration!(x_init, [1.0, 1.0])
set_velocity!(x_init, [0., 0.])

function collect_into!(data::Vector{<:LearningMPC.Sample})
    empty!(sample_sink)
    LearningMPC.randomize!(x0, x_init, 0.1, 0.5)
    results = LCPSim.simulate(x0, 
        dagger_controller,
        robot.environment, mpc_params.Δt, 100, 
        mpc_params.lcp_solver;
        termination=termination);
    append!(data, sample_sink.samples)
end

all_losses(net, dataset) = (LearningMPC.training_loss(net, dataset),
                            LearningMPC.validation_loss(net, dataset))

In [ ]:
LearningMPC.randomize!(x0, x_init, 0.1, 1.0)
results = LCPSim.simulate(x0, 
    lqrsol,
    robot.environment, mpc_params.Δt, 50, mpc_params.lcp_solver);

In [ ]:
LearningMPC.playback(basevis[:robot], results, mpc_params.Δt)

In [ ]:
losses = Tuple{Float64, Float64}[]
snapshots = LearningMPC.Snapshot{Float64}[]

@showprogress for i in 1:15
    for i in 1:2
        collect_into!(dataset.training_data)
    end
    collect_into!(dataset.testing_data)
    collect_into!(dataset.validation_data);
    
    for i in 1:5
        Nets.adam_update!(net.params.data, updater, loss, 
            LearningMPC.features.(dataset.training_data), adam_opts);
        push!(losses, all_losses(net, dataset))
    end
    
    push!(snapshots, LearningMPC.Snapshot(net.params.data, net))
    
    jldopen("hopper-$gradient_sensitivity.jld2", "w") do file
        file["dataset"] = dataset
        file["snapshots"] = snapshots
    end
    
    plt = plot(first.(losses), label="training")
    plot!(plt, last.(losses), label="validation")
    ylims!(plt, (0, ylims(plt)[2]))
    display(plt)
end

In [ ]:
LearningMPC.randomize!(x0, x_init, 0.1, 0.5)
results = LCPSim.simulate(x0, 
    net_controller,
    robot.environment, mpc_params.Δt, 200, mpc_params.lcp_solver);

In [ ]:
LearningMPC.playback(basevis[:robot], results, mpc_params.Δt)